In [ ]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Download latest version
path = kagglehub.dataset_download("yasserh/titanic-dataset")

df = pd.read_csv("Titanic-Dataset.csv")

df['Sex'] = df['Sex'].map({'male':0, 'female':1})
df['Age'] = pd.to_numeric(df['Age'], errors='coerce').fillna(df['Age'].median())

contingency_class = pd.crosstab(df['Pclass'], df['Survived'])
chi2_class, p_class, dof_class, _ = stats.chi2_contingency(contingency_class)
print(contingency_class)
print(f"Chi-square: {chi2_class:.3f}  p-value: {p_class:.6f}")

survive_rate_class = contingency_class[1] / contingency_class.sum(axis=1)
labels = ["1st Class", "2nd Class", "3rd Class"]
values = survive_rate_class.values
plt.bar(labels, values)
plt.title("Survival Rate by Passenger Class")
plt.ylabel("Survival Rate")
plt.show()

# Man vs Woman Chi-squared test (Who survives?)
# Survival rate vs pass. class
# Age outliers 

df

Survived    0    1
Pclass            
1          80  136
2          97   87
3         372  119
Chi-square: 102.889  p-value: 0.000000


AttributeError: module 'matplotlib' has no attribute 'bar'